<a href="https://colab.research.google.com/github/EloneSampaio/postgraduate/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning com TensorFlow



###  Introdução ao TensorFlow e Keras

TensorFlow: Um framework de código aberto para aprendizado de máquina, amplamente utilizado para redes neurais profundas.

Keras: API de alto nível em TensorFlow que facilita a construção de redes neurais.

Exemplo Prático: Transfer Learning com ResNet50 no TensorFlow

In [ ]:
#  Importando as Bibliotecas:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
#Carregando o Modelo Pré-treinado
model =  ResNet50(weights='imagenet')


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Carregando e Processando a Imagem
img_path =  '/content/drive/MyDrive/Colab Notebooks/rinocerontes.jpg'
img = image.load_img(img_path, target_size=(224,224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis= 0)
img_array = preprocess_input(img_array)

In [ ]:
# Fazendo Previsões
preds = model.predict(img_array)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [ ]:
# Resultado
print('Predict: ', decode_predictions(preds, top=5)[0])

Predict:  [('n02437312', 'Arabian_camel', 0.7455686), ('n02410509', 'bison', 0.05252746), ('n02504013', 'Indian_elephant', 0.050691888), ('n02397096', 'warthog', 0.048653632), ('n01704323', 'triceratops', 0.017607134)]


# Transfer Learning com PyTorch



###  Introdução ao Pytorch

Pytorch: Um framework de código aberto flexível e dinâmico, usado para redes neurais e aprendizado profundo. Permite uma maior personalização durante a execução do modelo.


### Exemplo prático: Transfer Learning com ResNet18 no Pytorch

ResNet-18 is a pretrained model that has been trained on a subset of the ImageNet database. The model is trained on more than a million images, and can classify images into 1000 object categories (e.g. keyboard, mouse, pencil, and many animals).

Passo a passo do exemplo
 - ResNet18: Aqui estamos usando o ResNet18 pré-treinado.
 - Preprocessamento da imagem: A imagem é redimensionada para 224x224 (o tamanho esperado pela ResNet), convertida para tensor, e normalizada.
 - Fazendo a previsão: O modelo retorna as pontuações, e torch.max(output, 1) retorna o índice da classe com a pontuação mais alta.
 - Mapeamento do índice para a label: Usamos a lista de classes para encontrar a label correspondente ao índice previsto.

In [28]:
# Importar bibliotecas

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [29]:
# Carregando o modelo pré-treinado
model = models.resnet18(pretrained=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
# Carregar e processar imagem
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])])

In [30]:
img_path = Image.open('/content/drive/MyDrive/Colab Notebooks/rinocerontes.jpg')

img_tensor = transform(img_path).unsqueeze(0) # Adicionar uma dimensão para o batch

In [31]:
# Fazendo previsões

# Desativar o cálculo do gradiente durante a inferência
with torch.no_grad():
    output = model(img_tensor)

# Obter o índice da previsão
_,predicted = torch.max(output,1)


In [32]:
# Previsão realizada
print('Predicted: ', predicted.indices)

Predicted:  <built-in method indices of Tensor object at 0x7d5d61bd7880>


In [33]:
# Recuperar a label correspondente ao índice previsto
import json
import urllib
# URL do arquivo JSON que contém as labels do ImageNet
url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = urllib.request.urlopen(url)
classes = json.loads(response.read().decode())
# Recuperar a label correspondente ao índice previsto
predicted_label = classes[predicted.item()]

print(f'Predicted label: {predicted_label}')


Predicted label: pig


In [17]:
print(output)

tensor([[ 4.8466e-01, -3.2300e+00,  1.2044e+00, -7.8125e-02, -1.2534e-01,
          1.4169e+00, -1.3278e+00,  7.7406e-01,  1.7931e-01,  1.9024e+00,
         -2.6394e+00, -4.0846e+00, -3.7236e+00, -3.7182e+00, -3.9239e+00,
         -2.5649e+00, -2.9976e+00, -3.1705e+00, -1.3188e+00, -3.6078e+00,
         -1.6679e+00, -1.9100e+00, -4.3515e-01,  2.4725e+00, -2.9689e+00,
          1.6577e+00,  1.2891e+00, -2.8923e-01,  1.4039e+00,  4.1984e+00,
         -4.7194e-01, -1.1926e+00,  7.6402e-01,  4.6476e+00,  4.4636e+00,
          3.1636e+00,  2.1567e+00,  7.8598e-01,  2.4532e+00,  6.5441e+00,
         -4.4711e-01,  8.8483e-01,  2.1509e+00,  1.8787e+00, -8.1494e-01,
          4.0567e+00, -8.0972e-01,  4.3191e+00,  8.0973e+00,  2.0966e+00,
         -6.9470e-01,  1.0090e+01, -3.4315e+00, -1.9734e+00, -5.4534e-01,
         -2.8100e+00, -2.2196e+00, -2.9940e+00, -3.5437e+00, -3.1287e+00,
         -1.4109e-02,  7.4608e-01,  1.1659e+00, -3.4897e-01,  8.1497e-02,
         -7.2556e-01,  1.8144e+00,  5.